In [8]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
import pandas as pd
from selenium.webdriver.common.by import By

path = 'C:/Users/joaop/OneDrive/Documentos/interfaces_web_final_proj/scraping/chromedriver.exe'

In [9]:
# Cria uma instância do Service com o caminho do chromedriver
service = Service(executable_path=path)
# Inicializa o driver passando o serviço
driver = webdriver.Chrome(service=service)

month = 11
year = 2024
day_i = 1
day = day_i

# URLs a fazer scraping
urls = []
for day in range(day, day+7):
    urls.append(f"https://www.basketball-reference.com/friv/dailyleaders.fcgi?month={month}&day={day}&year={year}&type=all")

# Criar uma lista para armazenar os dados
data = []

# Aceder a cada URL e realizar o scraping
for i, url in enumerate(urls):
    driver.get(url)
    time.sleep(2)  # Esperar 2 segundos para a página carregar
    
    # Obter o dia atual
    day_f = day_i
    current_day = day_f + i  

    try:
        # Localizar a tabela na página
        table = driver.find_element(By.XPATH, '//table')  
        
        # Extrair os dados da tabela
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            # Verificar se há colunas na linha
            if len(cols) > 0:
                row_data = [col.text for col in cols]
                date = f"{year}-{month:02d}-{current_day:02d}"  
                row_data.append(date)
                data.append(row_data) 
                print(row_data)  

    except Exception as e:
        print(f"Erro ao acessar a tabela na URL {url}: {e}")

# Criar um DataFrame com os dados recolhidos
columns = ["Player", "Tm", "vs", "Opp", "W/L", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "+/-", "GmSc", 'Date']
df = pd.DataFrame(data, columns=columns)

print(df)

# Fechar o driver
driver.quit()


['Anthony Davis', 'LAL', '@', 'TOR', 'W', '35:43', '14', '20', '.700', '0', '0', '', '10', '11', '.909', '1', '10', '11', '2', '3', '2', '3', '0', '38', '+7', '35.7', '2024-11-01']
['Nikola Vučević', 'CHI', '@', 'BRK', 'L', '33:57', '10', '11', '.909', '3', '4', '.750', '5', '5', '1.000', '3', '8', '11', '3', '1', '1', '0', '0', '28', '-7', '32.6', '2024-11-01']
['Zion Williamson', 'NOP', '', 'IND', 'W', '36:54', '14', '20', '.700', '1', '2', '.500', '5', '8', '.625', '1', '5', '6', '10', '2', '0', '3', '3', '34', '+17', '31.4', '2024-11-01']
['Nikola Jokić', 'DEN', '@', 'MIN', 'L', '39:42', '8', '16', '.500', '2', '3', '.667', '8', '10', '.800', '2', '7', '9', '13', '3', '1', '3', '1', '26', '+6', '30.1', '2024-11-01']
['Aaron Gordon', 'DEN', '@', 'MIN', 'L', '38:03', '11', '18', '.611', '5', '7', '.714', '4', '5', '.800', '4', '7', '11', '2', '1', '0', '0', '2', '31', '+8', '28.9', '2024-11-01']
['Jalen Brunson', 'NYK', '@', 'DET', 'W', '30:40', '14', '22', '.636', '6', '8', '.750', 

In [15]:
# Processamento da tabela
df = df[['Player', 'Tm', 'PTS', 'TRB', 'AST', 'Date']]

df.fillna({
    'Player': 'Desconhecido',  # Substitui valores nulos em 'Player' por 'Desconhecido'
    'Tm': 'Desconhecida',      # Substitui valores nulos em 'Tm' por 'Desconhecida'
    'PTS': 0,                  # Substitui valores nulos em 'PTS' por 0
    'TRB': 0,                  # Substitui valores nulos em 'TRB' por 0
    'AST': 0,                  # Substitui valores nulos em 'AST' por 0
    'Date': pd.to_datetime('today') # Substitui valores nulos em 'Date' pela data atual
}, inplace=True)

df['Date'] = pd.to_datetime(df['Date']).dt.date

# Seleciona as colunas do DataFrame que temos no modelo de dados Player
df = df[['Player', 'Tm', 'PTS', 'TRB', 'AST', 'Date']]

In [16]:
df

,Player,Tm,PTS,TRB,AST,Date
0,Anthony Davis,LAL,38,11,2,2024-11-01
1,Nikola Vučević,CHI,28,11,3,2024-11-01
2,Zion Williamson,NOP,34,6,10,2024-11-01
3,Nikola Jokić,DEN,26,9,13,2024-11-01
4,Aaron Gordon,DEN,31,11,2,2024-11-01
...,...,...,...,...,...,...
1141,Rayan Rupert,POR,0,0,0,2024-11-07
1142,Tyler Smith,MIL,0,0,0,2024-11-07
1143,Gary Trent,MIL,2,0,0,2024-11-07
1144,Isaiah Collier,UTA,0,0,1,2024-11-07


In [17]:
# Guarda o df num csv
df.to_csv('dailyleaders.csv', index=False)